In [ ]:
# Run this every time you open the spreadsheet
%load_ext autoreload
%autoreload 2
from collections import Counter
import lib
import random

## Exercise: Add bigram capabilities to the classifier!

So far our Naive Bayes classifier scores an Average F1 score of 66.9% on the test set.
Let's see if we can improve on that by incorporating bigrams!


In [ ]:
def add_bigrams(tweet):
    # Currently, tweet has an attribute called tweet.tokenList which is a list of tokens.
    # You want to add a new attribute to tweet called tweet.bigramList which is a list of bigrams.
    # Each bigram should be a pair of strings. You can define the bigram like this: bigram = (token1, token2).
    # In Python, this is called a tuple. You can read more about tuples here: https://www.programiz.com/python-programming/tuple

    ##### YOUR CODE STARTS HERE #####

    
    
    ##### YOUR CODE ENDS HERE #####


sandy_tweets, sandy_test_tweets = lib.read_data()
haiti_tweets, haiti_test_tweets = lib.read_haiti_data()

for tweet in sandy_tweets+sandy_test_tweets+haiti_tweets+haiti_test_tweets:
    add_bigrams(tweet)
print("Checking if bigrams are correct...")
for tweet in sandy_tweets+sandy_test_tweets+haiti_tweets+haiti_test_tweets:
    assert tweet._bigramList==tweet.bigramList, "Error in your implementation of the bigram list!"
print("Bigrams are correct.\n")


In [ ]:
# Let's take a look at the data distribution

print('# sandy:', len(sandy_tweets))
print('# sandy test:', len(sandy_test_tweets))
lib.class_pie_chart(tweets)

print()

print('# haiti:', len(haiti_tweets))
print('# haiti test:', len(haiti_test_tweets))
lib.class_pie_chart(test_tweets)


## Re-run the classifier and get evaluation score

This notebook uses our implementation of the Naive Bayes classifier, but it's very similar to what you implemented yesterday. If you're interested in the details, take a look at the `learn_nb` and `classify_nb` functions in `lib.py` in the `AI4ALL2018` directory.

For this exercise, let's use the Sandy dataset.

In [ ]:
# tweets, test_tweets = lib.read_data()
tweets, test_tweets = sandy_tweets, sandy_test_tweets
prior_probs, token_probs = lib.learn_nb(tweets)
predictions = [(tweet, lib.classify_nb(tweet, prior_probs, token_probs)) for tweet in test_tweets]
labels, precisions, recalls, f1s = lib.evaluate(predictions, has_return=True)

## Inspecting the Classifier

After implementing and training a classifier, you often want to inspect what kind of things it has learned and how it is making predictions on individual examples. This can help you make sure that you implemented everything correctly and it might give you ideas on how to further improve the classifier.

### Most discriminative words

Let's first look again at the most discriminative words for each category, i.e. the words that maximize P(category|word), for each category.

In [ ]:
lib.most_discriminative(tweets, token_probs, prior_probs)

These five lists show you which words are most predictive of the five categories. For example, the word _bottled_ is a very strong indicator that the tweet is about water or the word _canned_ is a very strong indicator that the tweet is about food.

Many of you used several of these words in your rule-based classifiers in week 1. It's reassuring (and exciting!) to see that the Naive Bayes classifier learned that these words are good indicators of the categories as well.


### Confusion matrix

Another useful type of visualization is a so-called confusion matrix. A confusion matrix shows you for each true category _c_ how many of the tweets in _c_ were classified into the five different categories. (In this way it tells you which categories are "confused" for others by the classifier).

In [ ]:
lib.show_confusion_matrix(predictions)

In the matrix, the **rows** correspond to the **true category** and the **columns** correspond to the **predicted category**.

For example, this matrix shows you that of all the 79 tweets in the category _None_, 13 were incorrectly classified as _Energy_, 3 as _Food_, and 1 as _Medical_. 62 of them were actually correctly classified as _None_.

### Visualizing individual tweets

It can also be really useful to visualize the probabilities of each token in an individual tweet. This can help you understand why a classifier made a correct or wrong prediction. We've implemented a visualization for you so that you can use to inspect how the classifier works on individual tweets.

In [ ]:
# The following code visualizes a random tweet from the test data. 
# Hover your mouse over the words!

random_tweet = random.choice(test_tweets)
lib.visualize_tweet(random_tweet, prior_probs, token_probs)

The color of each word tells you for which category $P(\text{token} \mid \text{category})$ is the highest. When you move the mouse over a word, it shows you the actual values of $P(\text{token} \mid \text{category})$ for each category that the classifier uses to make its predictions.

You can also have the classifier make a prediction on your own tweets. Change the text in `my_tweet` below and run the cell below to see what the classifier would predict.

In [ ]:
my_tweet = "I urgently need some bottled water."

lib.visualize_tweet(lib.Tweet(my_tweet, "?", ""), prior_probs, token_probs)

## More plots
Next let's generate some plots to better understand our data and to create some material for our presentation ;)

In [ ]:
# Generate a pie chart for class distribution
sandy_tweets = tweets+test_tweets
lib.class_pie_chart(sandy_tweets)

Q: what can you say about our class distribution?

In [ ]:
# Draw a pie chart for performance metrics: precision, recall, f1
lib.metric_bar_chart(labels, precisions, recalls, f1s)

## Using historical data

A lot of the times, the most effective way to improve a machine learning system is to get more data, which can often be difficult or even infeasible.  
Fortunately for us though, since Hurricane Sandy happens two years later than the Haiti earthquake, we can use the data from the Haiti earthquake to improve the performance on Hurricane Sandy. Let's try it out:

In [ ]:
# Sandy alone
sandy_tweets, sandy_test_tweets = lib.read_data()
tweets = sandy_tweets
test_tweets = sandy_test_tweets
prior_probs, token_probs = lib.learn_nb(tweets)
predictions = [(tweet, lib.classify_nb(tweet, prior_probs, token_probs)) for tweet in test_tweets]
_, _, _, sandy_f1s = lib.evaluate(predictions, has_return=True)
print()

# Augment the training set with historical data from Haiti
tweets = haiti_tweets + haiti_test_tweets + sandy_tweets
# tweets = [tweet for tweet in haiti_tweets if tweet.category in ['Energy', 'Medical', 'Water']] + [tweet for tweet in haiti_test_tweets if tweet.category in ['Energy', 'Medical', 'Water']] + sandy_tweets
test_tweets = sandy_test_tweets
prior_probs, token_probs = lib.learn_nb(tweets)
predictions = [(tweet, lib.classify_nb(tweet, prior_probs, token_probs)) for tweet in test_tweets]
_, _, _, combined_f1s = lib.evaluate(predictions, has_return=True)
print()

print("Comparing the performance in the two cases:")
labels = ['Energy', 'Food', 'Medical', 'None', 'Water']
acc_f1_alone = 0
acc_f1_combined = 0
for (label, alone, combined) in zip(labels, sandy_f1s, combined_f1s):
    print('    {}: alone:{} / combined:{}'.format(label, alone, combined))
    acc_f1_alone += alone
    acc_f1_combined += combined
print('Avg: alone:{} / combined:{}'.format(acc_f1_alone/5, acc_f1_combined/5))

Q: Does adding more data help with the performance? Why or why not?

Let's try another way of adding in historical data.

In [ ]:
# Sandy alone
sandy_tweets, sandy_test_tweets = lib.read_data()
tweets = sandy_tweets
test_tweets = sandy_test_tweets
prior_probs, token_probs = lib.learn_nb(tweets)
predictions = [(tweet, lib.classify_nb(tweet, prior_probs, token_probs)) for tweet in test_tweets]
_, _, _, sandy_f1s = lib.evaluate(predictions, has_return=True)
print()

# Augment the training set with historical data from Haiti
# following the category distribution in Sandy

import random

haiti_tweets, haiti_test_tweets = lib.read_haiti_data()
all_haiti_tweets = haiti_tweets + haiti_test_tweets


sandy_count = Counter([tweet.category for tweet in sandy_tweets])
haiti_count = Counter([tweet.category for tweet in all_haiti_tweets])

# Determine the number of examples to sample from Haiti
sample_size = len(all_haiti_tweets)
sandy_priors = Counter()
for category in sandy_count:
    sandy_priors[category] = sandy_count[category] / len(sandy_tweets)
    sample_size = min(sample_size, haiti_count[category] / sandy_priors[category])
sample_size = int(sample_size)

print("********************************")
print("Adding {:d} examples from Haiti".format(sample_size))
print("********************************\n")
    

# Randomly sample examples from Haiti
haiti_sampled = []
for category in haiti_count:
    haiti_examples = [tweet for tweet in all_haiti_tweets if tweet.category == category]
    haiti_sampled += random.sample(haiti_examples, int(sample_size * sandy_priors[category]))
    
    
# Train a Naive Bayer classifier using both Sandy training set
# and sampled examples from Haiti
tweets = sandy_tweets + haiti_sampled
test_tweets = sandy_test_tweets
prior_probs, token_probs = lib.learn_nb(tweets)
predictions = [(tweet, lib.classify_nb(tweet, prior_probs, token_probs)) for tweet in test_tweets]
_, _, _, combined_f1s = lib.evaluate(predictions, has_return=True)
print()


print("Comparing the performance in the two cases:")
labels = ['Energy', 'Food', 'Medical', 'None', 'Water']
acc_f1_alone = 0
acc_f1_combined = 0
for (label, alone, combined) in zip(labels, sandy_f1s, combined_f1s):
    print('    {}: alone:{} / combined:{}'.format(label, alone, combined))
    acc_f1_alone += alone
    acc_f1_combined += combined
print('Avg: alone:{} / combined:{}'.format(acc_f1_alone/5, acc_f1_combined/5))

Q: Does adding more historical data help? Why or why not?

## Error analysis: Figuring out remaining errors

Often, one wants to know in which scenarios a classifier makes mistakes. This can be really useful when you want to improve your classifier.

In this exercise, try to break the Naive Bayes classifier. Use the cell above and try to come up with a tweet which should be classified as _Food_ but which is assigned a different category. Once you find such a tweet, use the visualization to figure out why the classifier gets this example wrong.

Repeat this exercise for all the other categories. Based on your observations, do you have any ideas on how to further improve the classifier?
